In [13]:
import pandas as pd
import numpy as np
import subprocess
from datetime import date
import calendar
import bandit.main as bd

In [14]:
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [15]:
def df_bandit_diff(df):
    dfgb = df.groupby(['Class'])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [16]:
currDate = date.today()
list = [[x, y] for x in range(1,28) for y in [30,45,60]]

In [17]:
df = pd.DataFrame.from_records(list).rename(index=str, columns={0: "Goal", 1: "seconds"})
df['Date'] = currDate
df['Complete'] = np.nan
df = df[['Date','Goal','seconds','Complete']]
sheet_info = pd.read_excel(io='BikeGoals.xlsx')
df = df.append(sheet_info).reset_index(drop=True)
df['Cohort'] = df['Goal']  // 7

In [18]:
subprocess.Popen('BikeGoals.xlsx', shell=True)

In [19]:
df['Value'] = df['Goal'] * df['seconds']
df['Score'] = np.where(df['Complete'] >= df['Goal'], df['Value'], 0)
df.loc[df['Complete'].isnull(), 'Score'] = np.NaN
df['Class'] = df['Goal'].map(str) + '-' + df['seconds'].map(str)
df = bd.reduce(df,'Class')

1 0.9975 1


In [20]:
df_quality = bd.get_quality(df['Score'])
df = df.join(df_quality)
df.tail(10)

,Date,Goal,seconds,Complete,Cohort,Value,Score,Class,Quality
72,2018-09-16,25,30,NaN,3,750,NaN,25-30,NaN
73,2018-09-16,25,45,NaN,3,1125,NaN,25-45,NaN
74,2018-09-16,25,60,NaN,3,1500,NaN,25-60,NaN
75,2018-09-16,26,30,NaN,3,780,NaN,26-30,NaN
76,2018-09-16,26,45,NaN,3,1170,NaN,26-45,NaN
77,2018-09-16,26,60,NaN,3,1560,NaN,26-60,NaN
78,2018-09-16,27,30,NaN,3,810,NaN,27-30,NaN
79,2018-09-16,27,45,NaN,3,1215,NaN,27-45,NaN
80,2018-09-16,27,60,NaN,3,1620,NaN,27-60,NaN
81,2018-09-14 00:00:00,1,30,1.0,0,30,30.0,1-30,0.5


In [21]:
df_class = df_bandit_class(df,'Cohort')
df_class

,Cohort,count,mean,var,total,donext
1,1,0,NaN,NaN,1,999.0
2,2,0,NaN,NaN,1,999.0
3,3,0,NaN,NaN,1,999.0
0,0,1,0.5,NaN,1,0.5


In [22]:
nextClass = df_class.iloc[0,0]
nextClass

1

In [23]:
if df['Complete'].isnull().sum() > 0:
    df_filtered = df[df['Cohort'] == nextClass]
else:
    df_filtered = df

In [24]:
df_bandit_diff(df_filtered)

,Class,count,mean,var,total,donext
0,10-30,0,NaN,NaN,0,999.0
11,13-60,0,NaN,NaN,0,999.0
19,9-45,0,NaN,NaN,0,999.0
18,9-30,0,NaN,NaN,0,999.0
17,8-60,0,NaN,NaN,0,999.0
16,8-45,0,NaN,NaN,0,999.0
15,8-30,0,NaN,NaN,0,999.0
14,7-60,0,NaN,NaN,0,999.0
13,7-45,0,NaN,NaN,0,999.0
12,7-30,0,NaN,NaN,0,999.0
